In [3]:
import spotipy
import json
import spotipy.oauth2 as oauth2
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm

In [2]:
# Import songs from dataset
# csv_songs = pandas.read_csv()
columns = ['artist_name', 'title']
million_csv = pd.read_csv('MillionSongDB.csv', skipinitialspace=True, usecols=columns)
# artist_names = ['claris', 'asdf', 'claris']
# track_names = ['colorful', 'asdf', 'step']
million_csv['spotify_id'] = 'n/a'
print(million_csv.iloc[[0]]['artist_name'][0])
print(million_csv.iloc[[0]]['title'][0])

Casual
I Didn't Mean To


In [133]:
artist_list = million_csv['artist_name']
song_list = million_csv['title']

In [4]:
# Spotify Credentials Creation

credentials = oauth2.SpotifyClientCredentials(
        client_id='',
        client_secret='')

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)

In [135]:
# Load the songs we have already found
found_songs = []
with open('found_song_ids_json.txt') as json_file:
    found_songs = json.load(json_file)
found_songs = pd.read_json(found_songs)
print(len(found_songs))

10000


This is the section where I find all the song id's, skip this is we have all the id's

In [126]:
failed_songs = []

# Grab the song information and return it's uri for gathering audio features
def find_song_id(artist_name, track_name):
#     print(artist_name)
#     print(track_name)
    results = sp.search(q='artist:'+str(artist_name)+' track:'+str(track_name), limit=1)
    if(results['tracks']['total'] == 0):
        failed_songs.append(str(str(artist_name) +";;;" + str(track_name)))
        return None
    else:
        track_id =  results['tracks']['items'][0]['id']
        return track_id

In [127]:
tids = found_songs['spotify_id'].values

In [128]:
song_missing = []

filler_song = find_song_id('claris', 'shiori')
# iterate over all tracks and artists
for i in tqdm(range(len(found_songs), len(artist_list))):
    
    artist_name = artist_list[i]
    track_name =  song_list[i]
    track_id = find_song_id(artist_name, track_name)  
#     time.sleep(.5)
    if(track_id != None):
        tids = np.append(tids, track_id)
        song_missing.append(0)
        million_csv['spotify_id'][i] = track_id
        
    else: 
        # I don't really have a good solution to this... input a dummy song and clear it out later
        tids = np.append(tids, filler_song)
        song_missing.append(zip(str(artist_name), str(track_name)))
        million_csv['spotify_id'][i] = 'None'
    
    found_songs = found_songs.append(million_csv.iloc[[i]], ignore_index=True, sort=False)


In [129]:
million_json = found_songs.to_json(orient='records')
with open('found_song_ids_json.txt', 'w') as outfile:
    json.dump(million_json, outfile)

In [130]:
with open('failed_list.txt', 'a') as f:
    for item in failed_songs:
        print(item)
        f.write("%s\n" % item)

Sébastien Roch;;;nan
Ramón Vargas;;;Granada
Cama;;;Fade Away
Missy Higgins;;;The Special Two (Australian Version)
Dwight Twilley;;;Tulsa Girl
Faces;;;My Fault (LP Version)
Dropkick Murphys;;;The Only Road (Album Version)
Jimmie Davis;;;Jimmie's Travellin' Blues
Mayday;;;Dave D. Blues (how To Make It Sting Like A Career) (Album Version)
The Marx Brothers;;;Un jour aux courses: Who's That Man
My Chemical Romance;;;You Know What They Do To Guys Like Us In Prison (Album Version)
Wednesday 13's Frankenstein Drag Queens From Planet 13;;;Plan 9 From Outer Space (Album Version)
Cookies;;;Yan Chin
DJ X-Change featuring Joey A;;;Drop Low
Peggy Lee;;;It Never Happened To Me
Suzi Quatro;;;Don't Break My Heart
Dropkick Murphys;;;Johnny_ I Hardly Knew Ya
Martin Sexton;;;Glory Bound (LP Version)
Devanic;;;You Live_ You Die
Corderoy & U4IC DJ'S;;;3 Spirit
Barry Künzel;;;Supervisor (Instrumental) (edition Tromo/Rückbank)
Patrick Saint-Eloi;;;An Plas'ay
Fun Lovin' Criminals;;;Love Unlimited (Remix) (Exp

Now we have all songs + song id's, Grab the features of the ones we have


In [143]:
song_missing = np.zeros(len(tids))
filler_song = find_song_id('claris', 'shiori')
filler_song_features = sp.audio_features(filler_song)

features = []
# Grab the audio features in sets
for index in tqdm(range(0, len(tids))):
    if(tids[index] == 'None'):
        song_missing[index] = 1
        features.append(filler_song_features)
    else:
        features.append(sp.audio_features(tids[index]))

retrying ...1secs
retrying ...1secs


In [ ]:
# features = []
# for tid_list in split_tids:
#     features.append(sp.audio_features(tid_list))
# features = sp.audio_features(tids)

In [151]:
# Go back through and remove the song features that we couldn't find
for index in range(0, len(song_missing)):
    if(song_missing[index]):
        # Change all features to nan
#         print(features[index])
        for item in features[index][0]:
            features[index][0][item] = np.nan

In [152]:
# Write Json file to an output for easier managing so I don't keep grabbing it

with open('spotify_features.txt', 'w') as outfile:
    json.dump(features, outfile)

In [166]:
# Some spotify songs actually don't have audio features so it returns a null, have to manually handle that
features = []
with open('spotify_features.txt') as json_file:
     features = json.load(json_file)

In [167]:
# Add the artist names back
for i in range(0, len(features)):
    features[i][0]['title'] = song_list[i]
    features[i][0]['artist_name'] = artist_list[i]

In [170]:
# Remove extraneous features from the song
# print(features)
for element in features:
    print(element)
    element[0].pop('analysis_url')
    element[0].pop('id')
    element[0].pop('uri')
    element[0].pop('track_href')
# print(features)

[{'uri': 'spotify:track:01TR6aAKrA2cI3Z0gnCOsu', 'speechiness': 0.429, 'type': 'audio_features', 'liveness': 0.106, 'key': 6, 'track_href': 'https://api.spotify.com/v1/tracks/01TR6aAKrA2cI3Z0gnCOsu', 'loudness': -10.508, 'duration_ms': 218933, 'valence': 0.553, 'artist_name': 'Casual', 'id': '01TR6aAKrA2cI3Z0gnCOsu', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/01TR6aAKrA2cI3Z0gnCOsu', 'title': "I Didn't Mean To", 'danceability': 0.751, 'mode': 0, 'energy': 0.549, 'acousticness': 0.063, 'tempo': 92.224, 'instrumentalness': 2.47e-05, 'time_signature': 4}]
[{'uri': 'spotify:track:4yPl1mK1oluIrCwI4HInPR', 'speechiness': 0.0326, 'type': 'audio_features', 'liveness': 0.0748, 'key': 9, 'track_href': 'https://api.spotify.com/v1/tracks/4yPl1mK1oluIrCwI4HInPR', 'loudness': -12.94, 'duration_ms': 147760, 'valence': 0.966, 'artist_name': 'The Box Tops', 'id': '4yPl1mK1oluIrCwI4HInPR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4yPl1mK1oluIrCwI4HInPR', 'title': 'Soul 

[{'uri': 'spotify:track:2DIvv50yb83BGbZVEt5yyI', 'speechiness': 0.0928, 'type': 'audio_features', 'liveness': 0.0908, 'key': 6, 'track_href': 'https://api.spotify.com/v1/tracks/2DIvv50yb83BGbZVEt5yyI', 'loudness': -6.672, 'duration_ms': 145040, 'valence': 0.499, 'artist_name': 'Buzzcocks', 'id': '2DIvv50yb83BGbZVEt5yyI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2DIvv50yb83BGbZVEt5yyI', 'title': 'Get On Our Own', 'danceability': 0.139, 'mode': 1, 'energy': 0.923, 'acousticness': 0.00151, 'tempo': 171.587, 'instrumentalness': 5.18e-05, 'time_signature': 4}]
[{'uri': nan, 'speechiness': nan, 'type': nan, 'liveness': nan, 'key': nan, 'track_href': nan, 'loudness': nan, 'duration_ms': nan, 'valence': nan, 'artist_name': 'David Zinman;Pittsburgh Symphony Orchestra', 'id': nan, 'analysis_url': nan, 'title': 'Concerto for Orchestra (Zoroastrian Riddles) (1996)/3. Adagio non troppo', 'danceability': nan, 'mode': nan, 'energy': nan, 'acousticness': nan, 'tempo': nan, 'instrumen

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [171]:
# Write Json file to an output for easier managing so I don't keep grabbing it

with open('data_cleaned_with_songs.txt', 'w') as outfile:
    json.dump(features, outfile)

In [ ]:
# Convert the desired json file to a pandas dataframe

data = []
with open('data_cleaned.txt') as json_file:
    data = json.load(json_file)
df = json_normalize(data)